<a href="https://colab.research.google.com/github/DomiStryj/MHE/blob/master/Route_Inspection_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem chińskiego listonosza/Route Inspection Problem

In [59]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg texlive texlive-latex-extra texlive-fonts-extra texlive-latex-recommended texlive-science tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade
!pip install --upgrade plotly
!pip install jupyter-dash

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 1s (270 kB/s)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
34 packages can be upgraded. Run 'apt list --upgradable' to see them.
Read

# Opis problemu

Problem chińskiego listonosza polega na znalezieniu najkrótszej trasy do przejścia dla listonosza. Punktem startowym i kończowym jest poczta, miejsca, do których listonosz ma się udać z przesyłkami to wierzchołki grafu, a każda z ulic to krawędź.
Droga listonosza powinna wyglądać następująco:
1. listonosz odbiera przesyłki na poczcie
2. listonosz zanosi przesyłki do odbiorców, czyli musi dotrzeć do każdego z wierzchołków grafu, co najmniej jeden raz
3. listonosz kończy swoją trasę na poczcie.

Problem należy do problemów np, tylko wtedy, gdy graf nie zawiera cyklu Eulera.

Rozwiązaniem zadania jest znalezienie drogi, której suma wag krawędzi cyklu jest najmniejszą sumą wag krawędzi wszystkich takich cykli w grafie.

# Implementacja

# Przykład grafu, który można wykorzystać do wykonania zadania

Do wykonania zadania potrzebny będzie graf bez cyklu Eulera, czyli graf nieskierowany. Wierzchołek startowy i końcowy grafu to poczta, wierzchołki to skrzyżowania ulic, a ulice to krawędzie, które posiadają swoje wagi.



```
                   3
        (a)-----------------(b)
     1 /  |                  |  \1
      /   |                  |   \
     (c)  | 5               6|   (d)
      \   |                  |   /
     2 \  |         4        |  /1
        (e)------------------(f)
       
       a - b - d - f - d - b - f - e - c - a - c - e - a
```



In [2]:
import ast
with open("graf.txt", 'r') as f:
    punkty, odcinki, dlugosci = map(ast.literal_eval, f.readlines())

#Funkcja celu

Na początku została wyznaczona funkcja celu, mająca za zadanie określenie długości drogi, którą musi pokonać listonosz.

In [3]:
import numpy as np
import manim
import ast
import copy
import math
import random

Manim Community v0.17.3

In [4]:
def funkcjaCelu(rozwiazanie, punkt, odcinki, dlugosci):
    suma = 0
    tmp = []
    try:
        for i in range(0, len(rozwiazanie), 1): # na podstawie kolejności odwiedzanych punktów przydzieliłam pasujące do nich odcinki
            odc = punkt[rozwiazanie[i] - 1] + punkt[rozwiazanie[i + 1] - 1]
            # print(odc)
            tmp.append(odc)      # wszystkie odcinki znajdują się w tablicy tmp
    except IndexError:
        pass
    # print(tmp)
    for x in range(0, len(tmp), 1):     # dodajemy długości odcinków na podstawie tablicy tmp
        for y in range(0, len(odcinki), 1):
            if tmp[x] == odcinki[y]:
                suma += dlugosci[y]
    return suma
  
print(funkcjaCelu([1, 3, 5, 6, 4, 2, 1],punkty,odcinki,dlugosci))

12


# Najlepszy sąsiad

Do zastosowania algorytmu symulowanego wyżarzania będzie konieczne wyznaczenie najlepszego sąsiada. 

In [5]:
def losowySasiad(roz, punkty, odcinki):
    dl_max_roz = len(roz)   # maks. dł. rozwiązania
    # print("długść tablicy ", dl_max_roz)
    # print("rozwiązanie", roz)
    ptk = int(random.randint(0, len(roz) - 1)) #losuje ptk z zakresu rozwiązania
    # print("wylosowana liczba", ptk)
    # print("miejsce tablicy z indeksem wylo. liczby", roz[ptk])
    del roz[ptk + 1:]   # kasuję tablice od wybranego ptk
    # print("po usunięciu", roz)
    for k in range(len(roz), dl_max_roz, 1):
        punkt = punkty[roz[-1] - 1] #sprawdzam ptk z konca skasowanej tablicy
        # print(punkt)
        kolejne_wierzch = []
        # print(punkt)
        for i in range(0, len(odcinki), 1):  # wyznaczam dostępne wierzchołki na podst. ptk z końca tablicy
            if odcinki[i][:1] == punkt:
                kolejne_wierzch.append(odcinki[i][1:])
        # print(kolejne_wierzch)
        p = int(random.uniform(0, len(kolejne_wierzch))) #losujemy nowy wierzchołek dostępny z listy
        for j in range(0, len(punkty), 1):          #dopisuje nowy wierzchołek
            if punkty[j] == kolejne_wierzch[p]:
                roz.append(j + 1)
    # print("nowa tablica", roz)
    return roz

# Losowy sąsiad

W kolejnym etapie ważnym jest, aby wylosować najlepszego sąsiada, czyli wierzchołek (dom), który ma odwiedzić listonosz. Najlepszy sąsiad jest równy wierzchołkowi, do którego dotarcia listonosz pokonuje najkrótszą drogę. 
Losowy sąsiad jest elementem wykonania algorytmu wspinaczkowego.

In [6]:
def najlepszySasiad(cel, roz):  # cel, roz, punkty, odcinki
    najlepszy_wynik = roz
    dl_max_roz = len(roz)

    # print(roz)
    tym = roz
    for ptk in range(1, len(roz) - 1):
        # print(ptk)
        del tym[ptk:]
        # print("rozwiązanie w pętli ", tym)
        for k in range(len(roz), dl_max_roz, 1):
            punkt = punkty[roz[-1] - 1]
            kolejne_wierzch = []
            for i in range(0, len(odcinki), 1):
                if odcinki[i][:1] == punkt:
                    kolejne_wierzch.append(odcinki[i][1:])
            p = int(random.uniform(0, len(kolejne_wierzch)))
            for j in range(0, len(punkty), 1):
                if punkty[j] == kolejne_wierzch[p]:
                    tym.append(j + 1)

            if (tym[0] == tym[len(tym) - 1]) and \
                    cel(tym) <= cel(najlepszy_wynik):
                najlepszy_wynik = tym
                # print("naj", najlepszy_wynik)
    return najlepszy_wynik

# Generowanie losowej kolejności 

W celu zastosowania algorytmu wspinaczkowego, kolejnym krokiem jest wygenerowania losowej kolejności odiedzania wierzchołów przez listonosza. Wraz ze znalezieniem losowego sąsiada, element generowania losowej kolejności jest potrzebny do wykonania algorytmu wspinaczkowego.

In [7]:
def generowanieLosowejKolejnoci(wielkosc_rozw, punkty, odcinki):
    losowa_kolejnosc = [1]    #tablica zaczyna się domyślnie od punktu startu a=1
    for k in range(0, wielkosc_rozw - 1, 1):
        punkt = punkty[losowa_kolejnosc[-1] - 1] #zmienna posiada informacje punktu w którym się znajdujemy
        kolejne_wierzch = []
        #print(punkt)
        for i in range(0, len(odcinki), 1):   #petla wyznacza kolejne wierzchołki
            if odcinki[i][:1] == punkt:
                kolejne_wierzch.append(odcinki[i][1:])
        #print(kolejne_wierzch)
        p = int(random.uniform(0, len(kolejne_wierzch))) #losujemy liczbę o wielkości tablicy w której znajdują się dostępne wierzchołki
        # print(kolejne_wierzch[p])
        # print(punkty)
        for j in range(0, len(punkty), 1):
            if punkty[j] == kolejne_wierzch[p]:  # dodajemy wylosowany wierzchołek do kolejki
                losowa_kolejnosc.append(j + 1)
        #print(losowa_kolejnosc)
    return losowa_kolejnosc

# Algorytm wspinaczkowy

In [8]:
import time
import psutil

In [18]:
def determistycznyWspinaczkowy(cel):
    max_dl_roz = len(odcinki)  # maksymalny rozmiar rozwiązania
    min_dl_roz = int((len(odcinki) / 2) - 1)  # najmniejszy rozmiar rozwiązania
    iteracje = 1000  # ilość literacji
    tablica = []  # lista punktów odwiedzanych w proponowanym rozwiązaniu
    historia=[[1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1]]
    najlepszy_wynik = [1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1] #początkowa permutacja
    while max_dl_roz >= min_dl_roz:
        for j in range(0, iteracje, 1):
            roz = generowanieLosowejKolejnoci(max_dl_roz, punkty, odcinki)
            nowe_roz = najlepszySasiad(cel, roz)
            # print(nowe_roz)
            for k in range(1, len(punkty) + 1):
                odw_ptk = k in nowe_roz
                # print(k, odw_ptk)
                if odw_ptk:
                    tablica.append(k)
            if (len(tablica) >= len(punkty)) and \
                    (nowe_roz[0] == nowe_roz[len(nowe_roz) - 1]) and \
                    cel(nowe_roz) <= cel(najlepszy_wynik):
                # print("rozwiązanie wynosi :", cel(nowe_roz))
                najlepszy_wynik = nowe_roz
                for i in historia:
                  if i == najlepszy_wynik:
                    break
                  if i != najlepszy_wynik:
                    historia.append(najlepszy_wynik)
                    break
                # print("nowy rozwiązaniem jest sekwencja :", najlepszy_wynik)
            # print("odwiedzone punkty", tablica)
            tablica = []
        max_dl_roz -= 1
    print(historia)
    return najlepszy_wynik
czas_start_aw=time.time()
czas_koniec_aw=time.time()-czas_start_aw
roz = determistycznyWspinaczkowy(lambda s: funkcjaCelu(s, punkty, odcinki, dlugosci))
print(roz)
print("Czas działania: ", czas_koniec_aw)
print("Zużycie procesora:", psutil.cpu_percent())
print(funkcjaCelu(roz,punkty,odcinki,dlugosci))
#print(historia)

[[1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1, 5, 1], [1, 5, 1, 3, 5, 1, 2, 1, 2, 4, 6, 5, 6, 4, 2, 1], [1, 3, 5, 3, 5, 3, 1, 3, 1, 2, 6, 4, 2, 1, 5, 1], [1, 3, 1, 5, 6, 5, 3, 1, 2, 4, 2, 1, 3, 5, 3, 1], [1, 3, 1, 2, 6, 4, 6, 4, 2, 1, 3, 5, 3, 1, 2, 1], [1, 2, 4, 2, 6, 4, 6, 4, 6, 4, 6, 5, 3, 5, 3, 1], [1, 3, 1, 2, 4, 2, 4, 2, 4, 2, 4, 6, 4, 6, 5, 1], [1, 3, 1, 3, 1, 2, 4, 6, 4, 6, 5, 6, 4, 2, 1], [1, 3, 1, 2, 4, 2, 4, 2, 4, 6, 5, 1, 3, 1], [1, 3, 5, 1, 2, 4, 6, 4, 2, 4, 2, 1, 3, 1], [1, 2, 4, 2, 4, 6, 4, 6, 5, 3, 1, 3, 1], [1, 2, 4, 6, 4, 6, 5, 3, 1, 3, 1, 3, 1], [1, 5, 6, 4, 6, 4, 2, 1, 3, 1], [1, 3, 1, 2, 4, 6, 5, 1, 3, 1], [1, 3, 5, 1, 2, 4, 6, 4, 2, 1], [1, 2, 1, 2, 4, 6, 5, 3, 1], [1, 3, 5, 6, 4, 2, 1, 3, 1], [1, 2, 4, 6, 5, 3, 1], [1, 3, 5, 6, 4, 2, 1], [1, 3, 5, 6, 4, 2, 1]]
[1, 3, 5, 6, 4, 2, 1]
Czas działania:  3.218650817871094e-05
Zużycie procesora: 14.1
12


# Symulowane wyżarzanie

In [10]:
def symulowaneWyzarzanie(cel, Temperatura):
    max_dl_roz = len(odcinki)  # maksymalny rozmiar rozwiązania
    min_dl_roz = int((len(odcinki) / 2) - 1)  # najmniejszy rozmiar rozwiązania
    iteracje = 1000  # ilość literacji
    tablica = []  # lista punktów odwiedzanych w proponowanym rozwiązaniu
    najlepszy_wynik = generowanieLosowejKolejnoci(max_dl_roz, punkty, odcinki)
    while max_dl_roz >= min_dl_roz:

        V = [najlepszy_wynik]
        for j in range(1, iteracje + 1, 1):
            roz = generowanieLosowejKolejnoci(max_dl_roz, punkty, odcinki)
            nowe_roz = losowySasiad(roz, punkty, odcinki)
            # print(nowe_roz)
            for k in range(1, len(punkty) + 1):
                odw_ptk = k in nowe_roz
                #print(k, odw_ptk)
                if odw_ptk:
                    tablica.append(k)
            if (len(tablica) == len(punkty)) and \
                    (nowe_roz[0] == nowe_roz[len(nowe_roz) - 1]) and \
                    cel(nowe_roz) <= cel(najlepszy_wynik):
                # print("rozwiązanie wynosi :", cel(nowe_roz))
                najlepszy_wynik = nowe_roz
                V.append(najlepszy_wynik)
                #print("nowy rozwiązaniem jest sekwencja :", najlepszy_wynik)
                #print("odwiedzone punkty", tablica)

            else:
                e = math.exp(- abs(cel(nowe_roz) - cel(najlepszy_wynik)) / Temperatura(j))
                u = random.uniform(0.0, 1.0)
                if (u < e) and \
                        (len(tablica) == len(punkty)) and \
                        (nowe_roz[0] == nowe_roz[len(nowe_roz) - 1]):
                    najlepszy_wynik = nowe_roz
                    V.append(najlepszy_wynik)
                    # print("jest ")
            tablica = []
            #print(V)
        max_dl_roz -= 1
    return min(V, key=cel)
czas_start_sw=time.time()
czas_koniec_sw=time.time()-czas_start_sw
roz = symulowaneWyzarzanie(lambda s: funkcjaCelu(s, punkty, odcinki, dlugosci),lambda k: 1000 / k)
print(roz)
print("Czas działania: ", czas_koniec_sw)
print("Zużycie procesora:", psutil.cpu_percent())
print(funkcjaCelu(roz,punkty,odcinki,dlugosci))

[1, 3, 5, 6, 4, 2, 1]
Czas działania:  2.09808349609375e-05
Zużycie procesora: 97.6
12


# Porównanie metod

Porównując działania obu metod ciężko jest wybrać lepszą, ponieważ wyniki działania obu z nich są bardzo porównywalne. Poprzez wykonanie około 20 prób można stwierdzić, że algorytm symulowanego wyżarzania jest minimalnie szybszy, ale zużywa więcej zasobów procesora. Jeśli chodzi o zużycie pamięci RAM wyniki są identyczne.

Algorytm wspinaczkowy- wyniki testowe:

In [46]:
print(roz)
print("Czas działania: ", czas_koniec_aw)
print("Zużycie procesora:", psutil.cpu_percent())
print("Zużycie RAMu w %: ", psutil.virtual_memory())

[1, 3, 5, 6, 4, 2, 1]
Czas działania:  3.218650817871094e-05
Zużycie procesora: 2.0
Zużycie RAMu w %:  svmem(total=13613326336, available=11926466560, percent=12.4, used=1345789952, free=7073275904, active=1098772480, inactive=4613009408, buffers=551047168, cached=4643213312, shared=7839744, slab=710078464)


Algorytm symulowanego wyżarzania- wyniki testowe:

In [42]:
print(roz)
print("Czas działania: ", czas_koniec_sw)
print("Zużycie procesora:", psutil.cpu_percent())
print("Zużycie RAMu w %: ", psutil.virtual_memory())

[1, 3, 5, 6, 4, 2, 1]
Czas działania:  2.09808349609375e-05
Zużycie procesora: 2.7
Zużycie RAMu w %:  svmem(total=13613326336, available=11924611072, percent=12.4, used=1347510272, free=7071674368, active=1098756096, inactive=4612665344, buffers=551047168, cached=4643094528, shared=7839744, slab=710021120)


# Wizualizacja

In [13]:
edges = [(0,1),(1,2),(2,3),(3,4),(4,5),(5,6),(6,0)]
vertices=[i for i in range(6)]
# Tworzenie pustego słownika reprezentującego graf
graph = {}

# Dodawanie krawędzi do grafu
for edge in edges:
  vertex1=edge[0]
  vertex2=edge[1]

  if vertex1 in graph:
    graph[vertex1].append(vertex2)
  else:
    graph[vertex1]=[vertex2]
  if vertex2 in graph:
    graph[vertex2].append(vertex1)
  else:
    graph[vertex2]=[vertex1]
for vertex, neighbors in graph.items():
  print(f"Wierzchołek {vertex}: {neighbors}")

  

Wierzchołek 0: [1, 6]
Wierzchołek 1: [0, 2]
Wierzchołek 2: [1, 3]
Wierzchołek 3: [2, 4]
Wierzchołek 4: [3, 5]
Wierzchołek 5: [4, 6]
Wierzchołek 6: [5, 0]


Tworzenie grafu polega na stworzeniu słownika "graph", który rysuje graf na podstawie podanych krawędzi, następnie wypisuje listę sąsiadów dla każdego wierzchołka.

In [25]:
from manim import *

In [15]:
edges

[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 0)]

In [57]:
import copy
import networkx as nx
import matplotlib.pyplot as plt

In [58]:
# Definiowanie krawędzi
edges = [(0,1),(1,2),(2,3),(3,4),(4,5),(5,0)]

# Tworzenie pustego grafu skierowanego
G = nx.DiGraph()

# Dodawanie krawędzi do grafu
G.add_edges_from(edges)

# Rysowanie grafu i zapisywanie sekwencji obrazów
fig = plt.figure(figsize=(6, 4), dpi=100)
pos = nx.spring_layout(G)
frames = []
for i in range(10):
    nx.draw_networkx(G, pos, node_color='lightblue', node_size=500)
    plt.axis('off')
    frames.append(fig)
    plt.clf()

# Zapisywanie sekwencji obrazów jako animację w formacie PNG
for i, frame in enumerate(frames):
    frame.savefig(f"frame_{i}.png")


<Figure size 600x400 with 0 Axes>